In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
import networkx as nx
import os
import warnings

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LassoCV, LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsRegressor

from xgboost import XGBRegressor
import xgboost as xgb


# 1.数据处理

In [3]:
data = pd.read_csv(r"D:\论文数据\2025.2 出口选择\20190519+问卷统计表.csv", encoding='utf-8') 
data = data.drop('序号', axis=1)
#data

In [4]:
rename_dict = {
    '性别': 'Gen',
    '年龄段': 'Age',
    '教育程度': 'Edu',
    '行动能力': 'Mob',
    '您乘坐客船的经历（不含本次）': 'PCE',
    '与您一起出行的人员数量': 'GS',
    '接受船舶疏散教育/训练的经历': 'ET',                     
    '听到疏散逃生警报，您会等待工作人员确认后再行动': 'WFS',
    '听到疏散逃生警报，您会立即疏散逃生': 'EI',
    '听到疏散逃生警报，您会观察其他人的动静再行动': 'OO',
    '假如听到疏散逃生警报，您会自行开门查看确认': 'PC',      
    '疏散逃生时，您会选择距离最近的出口': 'NE',  #
    '疏散逃生时，您会选择最熟悉的出口': 'FE',  #
    '疏散逃生时，您会选择跟着大多数人走': 'FM',  #
    '疏散逃生时，您会选择听从疏散指示或广播': 'FI',  #       
    '您会耐心排队等待': 'QP',
    '您会自行寻找其他出口': 'SOE',  
    '您会往前挤行': 'SF',
    '您会听从船员引导': 'OC',
    '贵重物品遗留，您是否会返回寻找': 'RIV',
    '家人遗留，您是否会返回寻找': 'RIF',
    '疏散过程中，您是否会协助他人进行疏散': 'HO',
    '疏散过程中，您是否会超越他人或向前挤行': 'OtO',
    '疏散逃生时，您是否会寻找同伴一起逃生': 'FC',
    '假如发生火灾，您是否会出现恐慌心理': 'EP',
    '疏散时，您是否会携带大件行李箱': 'CL',
    '疏散时，您是否会跟从团队中的临时领导者': 'FL'
}
data.rename(columns=rename_dict, inplace=True)

In [5]:
df = data
df_ec = df.drop(columns=['NE', 'FE', 'FM', 'FI'])
df_ne = df.drop(columns=['FE', 'FM', 'FI'])
df_fe = df.drop(columns=['NE', 'FM', 'FI'])
df_fm = df.drop(columns=['NE', 'FE', 'FI'])
df_fi = df.drop(columns=['NE', 'FE', 'FM'])

In [6]:
df_ec.columns

Index(['Gen', 'Age', 'Edu', 'Mob', 'PCE', 'GS', 'ET', 'WFS', 'EI', 'OO', 'PC',
       'QP', 'SOE', 'SF', 'OC', 'RIV', 'RIF', 'HO', 'OtO', 'FC', 'EP', 'CL',
       'FL'],
      dtype='object')

# 2.集成因果推断

In [7]:
# =======================
# 数据预处理
# =======================
# 前部变量（行为与心理因素）
front_vars = ['Gen', 'Age', 'Edu', 'Mob', 'PCE', 'GS', 'ET', 'WFS', 'EI', 'OO', 'PC',
       'QP', 'SOE', 'SF', 'OC', 'RIV', 'RIF', 'HO', 'OtO', 'FC', 'EP', 'CL',
       'FL']

# 后部变量（出口选择行为）
exit_vars = ['NE', 'FE', 'FM', 'FI']

In [8]:
# 计算互信息
mi_scores = {}
for exit_var in exit_vars:
    mi = mutual_info_classif(df[front_vars], df[exit_var], discrete_features='auto')
    mi_scores[exit_var] = dict(zip(front_vars, mi))

# 转换为DataFrame
mi_df = pd.DataFrame(mi_scores)

# 计算每个前部变量的平均影响力
mi_df["Mean_MI"] = mi_df.mean(axis=1)

# 按影响力排序
mi_df_sorted = mi_df.sort_values(by="Mean_MI", ascending=False)

print(mi_df_sorted)

           NE        FE        FM        FI   Mean_MI
PC   0.260047  0.209781  0.161722  0.131412  0.190740
QP   0.124216  0.127224  0.159045  0.292917  0.175851
OO   0.209364  0.178317  0.135709  0.106433  0.157456
WFS  0.181809  0.158733  0.141905  0.109891  0.148085
EI   0.181910  0.172366  0.148715  0.085770  0.147190
OC   0.117207  0.140884  0.078377  0.127573  0.116010
SOE  0.089438  0.111212  0.112625  0.098241  0.102879
SF   0.074033  0.097036  0.078704  0.081900  0.082918
FC   0.051598  0.069714  0.049209  0.078044  0.062141
OtO  0.068178  0.042249  0.044786  0.059655  0.053717
EP   0.046869  0.020227  0.059665  0.069919  0.049170
HO   0.075756  0.045221  0.037388  0.028617  0.046746
RIF  0.054854  0.059158  0.046048  0.024434  0.046123
RIV  0.041381  0.040443  0.031358  0.038559  0.037935
CL   0.045507  0.045691  0.009095  0.031615  0.032977
FL   0.034154  0.028437  0.028270  0.028614  0.029869
Age  0.000000  0.036236  0.025906  0.055782  0.029481
Mob  0.027178  0.000000  0.0

In [8]:
# 处理变量
Control_vars = ['Gen','Edu','Mob','GS','ET','FL' ]

# 处理变量
Treatment_vars = ['Age', 'PCE', 'WFS', 'EI', 'OO', 'PC',
       'QP', 'SOE', 'SF', 'OC', 'RIV', 'RIF', 'HO', 'OtO', 'FC', 'EP', 'CL']

# 结果变量（出口选择行为）
Outcome_vars = ['NE']

In [9]:
#因果森林 因果Boosting dml 

In [10]:
X_control = df[Control_vars]
X_treat = df[Treatment_vars]
Y_outcomes = df[Outcome_vars]

# 标准化控制变量
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_control), columns=Control_vars)

In [33]:
def fit_causal_models(Y, T, X_scaled, random_state=42):
    """运行三个因果模型并返回个体效应估计（稳定版）"""

    epsilon = 1e-3  # 防止分母过小
    warnings.filterwarnings("ignore")
    
    # ========================================
    # 1️⃣ 模型1：随机森林因果估计（Causal Forest近似）
    # ========================================
    rf = RandomForestRegressor(random_state=random_state)
    rf_param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, 20, None],
        'min_samples_leaf': [2, 5, 10]
    }
    rf_search = RandomizedSearchCV(rf, rf_param_grid, n_iter=5, cv=3, n_jobs=-1, random_state=random_state)
    rf_search.fit(X_scaled, Y)
    best_rf = rf_search.best_estimator_

    # 辅助模型预测 T
    rf_t = RandomForestRegressor(n_estimators=200, min_samples_leaf=10, random_state=random_state)
    rf_t.fit(X_scaled, T)

    y_resid = Y - best_rf.predict(X_scaled)
    t_resid = T - rf_t.predict(X_scaled)

    tau_cf = np.divide(y_resid, np.where(np.abs(t_resid)<epsilon, epsilon*np.sign(t_resid), t_resid))
    tau_cf = np.nan_to_num(tau_cf, nan=0.0, posinf=0.0, neginf=0.0)

    # ========================================
    # 2️⃣ 模型2：Causal Boosting（XGBoost残差法）
    # ========================================
    # 拟合Y和T
    xgb_y = xgb.XGBRegressor(n_estimators=500, max_depth=5, learning_rate=0.05,
                             objective='reg:squarederror', random_state=random_state)
    xgb_y.fit(X_scaled, Y)
    y_pred = xgb_y.predict(X_scaled)
    y_res = Y - y_pred

    xgb_t = xgb.XGBRegressor(n_estimators=500, max_depth=5, learning_rate=0.05,
                             objective='reg:squarederror', random_state=random_state)
    xgb_t.fit(X_scaled, T)
    t_pred = xgb_t.predict(X_scaled)
    t_res = T - t_pred

    tau_cb = np.divide(y_res, np.where(np.abs(t_res)<epsilon, epsilon*np.sign(t_res), t_res))
    tau_cb = np.nan_to_num(tau_cb, nan=0.0, posinf=0.0, neginf=0.0)

    # ========================================
    # 3️⃣ 模型3：DML近似
    # ========================================
    # 可以使用与 tau_cb 相同的残差方法，或者换 KNN
    knn_y = KNeighborsRegressor(n_neighbors=10)
    knn_y.fit(X_scaled, Y)
    y_pred_knn = knn_y.predict(X_scaled)
    y_res_knn = Y - y_pred_knn

    knn_t = KNeighborsRegressor(n_neighbors=10)
    knn_t.fit(X_scaled, T)
    t_pred_knn = knn_t.predict(X_scaled)
    t_res_knn = T - t_pred_knn

    tau_dml = np.divide(y_res_knn, np.where(np.abs(t_res_knn)<epsilon, epsilon*np.sign(t_res_knn), t_res_knn))
    tau_dml = np.nan_to_num(tau_dml, nan=0.0, posinf=0.0, neginf=0.0)

    return tau_cf, tau_cb, tau_dml

In [34]:
# -----------------------------
# Step 3. 集成逻辑：自动加权融合（基于RMSE）
# -----------------------------
def ensemble_effect(Y, tau_cf, tau_cb, tau_dml):
    """
    根据训练集上的RMSE自动计算权重并融合三种因果估计
    Y: 真实结果
    tau_cf, tau_cb, tau_dml: 各模型对训练集的个体效应预测
    """
    from sklearn.metrics import mean_squared_error

    # 计算每个模型的RMSE
    rmse_cf = np.sqrt(mean_squared_error(Y, tau_cf))
    rmse_cb = np.sqrt(mean_squared_error(Y, tau_cb))
    rmse_dml = np.sqrt(mean_squared_error(Y, tau_dml))

    # 根据RMSE反比设置权重（RMSE越小，权重越大）
    inv_rmse = np.array([1/rmse_cf, 1/rmse_cb, 1/rmse_dml])
    weights = inv_rmse / np.sum(inv_rmse)  # 归一化到 [0,1]

    # 加权融合
    tau_ens = weights[0]*tau_cf + weights[1]*tau_cb + weights[2]*tau_dml

    # 返回融合结果和权重
    return tau_ens, weights[0], weights[1], weights[2]


In [35]:
# ---------------------------------------------------------
# Step 4. 主循环：计算各结果与处理变量的ATE
# ---------------------------------------------------------
results = []  

for outcome in Outcome_vars:
    print(f"\n🔸 集成因果推断结果: {outcome}")
    Y = df[outcome].values

    for treat in Treatment_vars:
        T = df[treat].values

        # 拟合三个模型
        tau_cf, tau_cb, tau_dml = fit_causal_models(Y, T, X_scaled)

        # 加权集成
        tau_ens, w_cf, w_cb, w_dml = ensemble_effect(Y, tau_cf, tau_cb, tau_dml)

        # 汇总结果
        results.append({
            "Outcome": outcome,
            "Treatment": treat,
            "ATE_CF": np.mean(tau_cf),
            "ATE_CB": np.mean(tau_cb),
            "ATE_DML": np.mean(tau_dml),
            "ATE_Ensemble": np.mean(tau_ens),
            "Weight_CF": w_cf,
            "Weight_CB": w_cb,
            "Weight_DML": w_dml
        })

df_result = pd.DataFrame(results)
print("\n✅ 集成模型ATE汇总：")
print(df_result.head())



🔸 集成因果推断结果: NE

✅ 集成模型ATE汇总：
  Outcome Treatment    ATE_CF    ATE_CB   ATE_DML  ATE_Ensemble  Weight_CF  \
0      NE       Age -0.086412 -0.731176  0.116558     -0.036550   0.111443   
1      NE       PCE -2.995044 -2.115121  0.271666     -0.181901   0.071678   
2      NE       WFS -0.300264 -2.286899  0.236650      0.008757   0.219916   
3      NE        EI  0.281518  0.340859  0.366814      0.336871   0.295015   
4      NE        OO  0.644948 -0.755556  0.474902      0.300818   0.079490   

   Weight_CB  Weight_DML  
0   0.153925    0.734631  
1   0.091929    0.836393  
2   0.043517    0.736567  
3   0.184155    0.520831  
4   0.152464    0.768045  


In [37]:
# ---------------------------------------------------------
# Step 5. 保存结果
# ---------------------------------------------------------
save_path = r"D:\论文数据\2025.10 出口选择"
os.makedirs(save_path, exist_ok=True)
file_name = os.path.join(save_path, "ensemble_causal_results.csv")

df_result.to_csv(file_name, index=False, encoding='utf-8-sig')
print(f"\n📁 结果已保存至：{file_name}")


📁 结果已保存至：D:\论文数据\2025.10 出口选择\ensemble_causal_results.csv


# 3.MSTAN-BN

In [1]:
# =======================
# 假设 df 已经读入，目标列 'NE'
# =======================

# 将特征列转换为类别类型
target_col = 'NE'
feature_cols = [col for col in df_ne.columns if col != target_col]

for col in feature_cols + [target_col]:
    df_ne[col] = df_ne[col].astype('category')

X = df_ne[feature_cols]
Y = df_ne[target_col]


NameError: name 'df_ne' is not defined

In [ ]:
# =======================
# 计算条件互信息函数
# =======================
def conditional_mutual_info(X_i, X_j, Y):
    mi = 0
    for y_val, prob_y in Y.value_counts(normalize=True).items():
        idx = (Y == y_val)
        xi_subset = X_i[idx]
        xj_subset = X_j[idx]
        joint_counts = pd.crosstab(xi_subset, xj_subset)
        joint_prob = joint_counts / joint_counts.values.sum()
        xi_prob = joint_prob.sum(axis=1)
        xj_prob = joint_prob.sum(axis=0)
        for xi_val in joint_prob.index:
            for xj_val in joint_prob.columns:
                p_ij = joint_prob.at[xi_val, xj_val]
                if p_ij > 0:
                    mi += prob_y * p_ij * np.log(p_ij / (xi_prob[xi_val]*xj_prob[xj_val]))
    return mi


In [ ]:
# =======================
# 构建条件互信息矩阵 & 列表 L
# =======================
pairs = list(combinations(feature_cols, 2))
I_dict = {}
for Xi, Xj in pairs:
    I_dict[(Xi, Xj)] = conditional_mutual_info(X[Xi], X[Xj], Y)

In [ ]:
# 按互信息降序排列
L = sorted(I_dict.keys(), key=lambda x: I_dict[x], reverse=True)

# =======================
# 构建 TAN (MWST)
# =======================
# 1. 用NetworkX构建图
G = nx.Graph()
G.add_nodes_from(feature_cols)
for (Xi, Xj), w in I_dict.items():
    G.add_edge(Xi, Xj, weight=w)


In [ ]:
# 2. 最大加权生成树
MWST = nx.maximum_spanning_tree(G)


In [ ]:
# 3. 转为有向图，选择任意根节点（这里选第一个特征列）
root = feature_cols[0]
TAN_directed = nx.bfs_tree(MWST, root)
T_r = {child: None for child in feature_cols}
for parent, child in TAN_directed.edges():
    T_r[child] = parent


In [ ]:
# =======================
# 构建 STAN
# =======================
def learn_STAN(X, Y, T_r, L, k=2):
    S_r = T_r.copy()
    G_dir = nx.DiGraph()
    for child, parent in S_r.items():
        if parent is not None:
            G_dir.add_edge(parent, child)
        else:
            G_dir.add_node(child)
    parent_count = {node: 0 for node in X.columns}
    for child, parent in S_r.items():
        if parent is not None:
            parent_count[child] += 1
    for Xi, Xj in L:
        if G_dir.has_edge(Xi, Xj) or G_dir.has_edge(Xj, Xi):
            continue
        AH_i_j = conditional_mutual_info(X[Xi], X[Xj], Y)
        AH_j_i = conditional_mutual_info(X[Xj], X[Xi], Y)
        parent, child = (Xi, Xj) if AH_i_j > AH_j_i else (Xj, Xi)
        if parent_count[child] < k:
            G_dir.add_edge(parent, child)
            if nx.is_directed_acyclic_graph(G_dir):
                S_r[child] = parent
                parent_count[child] += 1
            else:
                G_dir.remove_edge(parent, child)
    return S_r

S_r = learn_STAN(X, Y, T_r, L, k=2)

In [ ]:
# =======================
# 输出结果
# =======================
print("TAN结构 (T_r):")
print(T_r)
print("\nSTAN结构 (S_r):")
print(S_r)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_stan_structure(S_r, title="STAN Structure"):
    """
    S_r: dict {child: parent}
    title: 图标题
    """
    G = nx.DiGraph()
    
    # 添加节点和边
    for child, parent in S_r.items():
        G.add_node(child)
        if parent is not None:
            G.add_edge(parent, child)
    
    # 位置布局
    pos = nx.spring_layout(G, seed=42)  # 弹簧布局
    plt.figure(figsize=(12, 8))
    
    # 绘制节点
    nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='lightblue')
    # 绘制边
    nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=20, edge_color='gray')
    # 绘制标签
    nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
    
    plt.title(title, fontsize=16)
    plt.axis('off')
    plt.show()

# ===== 绘制 STAN 结构 =====
plot_stan_structure(S_r, title="STAN Structure for NE")
